In [30]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv


In [51]:
!pip install scikit-learn==1.2.2
!pip install imbalanced-learn==0.10.1
!pip install xgboost
!pip install mlflow
!pip install dagshub
import dagshub
dagshub.init(repo_owner='gchit21',repo_name='ML2',mlflow=True)

Initialized MLflow to track repo "gchit21/ML2"

Repository gchit21/ML2 initialized!

In [66]:
#Imports
import mlflow
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from lightgbm import LGBMClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import (
    precision_score, recall_score, f1_score,
    precision_recall_curve, roc_auc_score, roc_curve, confusion_matrix
)
from category_encoders.woe import WOEEncoder
from sklearn.feature_selection import RFE
import matplotlib.pyplot as plt


In [33]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

In [34]:
transaction_df = pd.read_csv("/kaggle/input/ieee-fraud-detection/train_transaction.csv")
identity_df=pd.read_csv("/kaggle/input/ieee-fraud-detection/train_identity.csv")
transaction_test_df=pd.read_csv("/kaggle/input/ieee-fraud-detection/test_transaction.csv")
identity_test_df=pd.read_csv("/kaggle/input/ieee-fraud-detection/test_identity.csv")

In [35]:
identity_df.shape

(144233, 41)

In [36]:
mutual = [cols for cols in transaction_df.columns if cols ==identity_df.columns.any() ]

In [37]:
joined_df = transaction_df.merge(identity_df,on='TransactionID',how="left")
joined_test_df = transaction_test_df.merge(identity_test_df,on='TransactionID',how="left")
joined_test_df.columns = joined_test_df.columns.str.replace('-', '_', regex=True) # correcting the column names in the test dataset

In [38]:
joined_test_df.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,3663549,18403224,31.95,W,10409,111.0,150.0,visa,226.0,debit,170.0,87.0,1.0,NaN,gmail.com,NaN,6.0,6.0,0.0,0.0,3.0,4.0,0.0,0.0,6.0,0.0,5.0,1.0,115.0,6.0,419.0,419.0,27.0,398.0,27.0,NaN,NaN,NaN,NaN,418.0,203.0,NaN,NaN,NaN,409.0,T,T,F,NaN,NaN,F,T,T,T,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,47.950001,0.000000,0.000000,47.950001,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,47.950001,0.000000,0.000000,47.950001,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3663550,18403263,49.00,W,4272,111.0,150.0,vis

In [39]:
cat_info = ['userId','P_emaildomain','R_emaildomain','DeviceType','DeviceInfo','ProductCD','addr1', 'addr2',]
cat_card = ['card' + str(i) for i in range(1, 7)]
cat_m = ['M' + str(i) for i in range(1, 10)]
cat_id = ['id_' + str(i) for i in range(12, 39)]
cat_columns = cat_info + cat_card + cat_m + cat_id
num_columns = [col for col in joined_df.columns if joined_df[col].dtype != 'object' and col != 'isFraud' and col !='TransactionID']

In [41]:
dropped_df = joined_df.drop(columns=cols_to_drop,axis=1) 
dropped_test_df = joined_test_df.drop(columns=cols_to_drop,axis=1)
X=dropped_df.drop(["TransactionID","isFraud"],axis=1)
y=dropped_df["isFraud"]

In [42]:
#Split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)
sampler = RandomUnderSampler(random_state=42)

# Column dropper

In [69]:
class DropColumns(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.75):
        self.columns_to_drop = None
        self.threshold = threshold

    def fit(self, X, y=None):
        missing_cols = [col for col in X.columns if X[col].isna().any()]
        high_missing_cols = [
            col for col in missing_cols 
            if (X[col].isna().mean()) > self.threshold
        ]
        
        self.columns_to_drop = high_missing_cols
        return self

    def transform(self, X):
        if self.columns_to_drop is None:
            raise ValueError('Needs Fitting !')
        
        return X.drop(columns=self.columns_to_drop)

# Categorial column encoder

In [44]:
class CatEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=2):
        self.threshold = threshold
        self.binary_cols = []
        self.multi_cols = []
        self.encoder = None

    def fit(self, X, y):
        X = X.copy()
        
        for col in X.columns:
            if X[col].dtype == 'object':
                non_rep = X[col].nunique()
                if non_rep <= self.threshold:
                    self.binary_cols.append(col)
                else:
                    self.multi_cols.append(col)

        self.encoder = WOEEncoder(cols=self.multi_cols)
        self.encoder.fit(X[self.multi_cols], y)
        
        return self

    def transform(self, X):
        X = X.copy()
        if self.multi_cols:
            X_woe = self.encoder.transform(X[self.multi_cols])
            X[self.multi_cols] = X_woe

        if self.binary_cols:
            X = pd.get_dummies(X, columns=self.binary_cols, drop_first=True, dtype=int)
        
        return X

# Corralation filter

In [61]:
class CorrelationTargetFilter(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.85):
        self.threshold = threshold
        self.to_remove_ = []

    def fit(self, X, y=None):
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input must be a pandas DataFrame.")

        abs_corr = X.corr().abs()
        triu_mask = np.triu(np.ones(abs_corr.shape), k=1).astype(bool)
        upper_triangle_df = pd.DataFrame(triu_mask, index=abs_corr.index, columns=abs_corr.columns)

        correlated_features = set()
        for i, col in enumerate(abs_corr.columns):
            high_corr = abs_corr[col][upper_triangle_df[col]].loc[lambda s: s > self.threshold].index
            correlated_features.update(high_corr)

        self.to_remove_ = list(correlated_features)
        print(f"Found {len(self.to_remove_)} features with high correlation to drop.")
        return self

    def transform(self, X):
        if self.to_remove_ is None:
            raise RuntimeError("This transformer has not been fitted yet.")
        return X.drop(columns=self.to_remove_, errors="ignore")



# NA Value Filler

In [46]:
class FillNa(BaseEstimator, TransformerMixin):
    def __init__(self, num_cols, cat_cols):
        self.num_cols = num_cols
        self.cat_cols = cat_cols
        self.values = {}

    def fit(self, X, y=None):
        for col in self.num_cols:
            if col in X.columns:
                val = X[col].median()
                X[col] = X[col].fillna(val)
                self.values[col] = val

        for c in self.cat_cols:
            if c in X.columns:  
                if X[c].dtype in ['int64', 'float64']:
                    X[c] = X[c].astype(str)
                X[c] = X[c].fillna('NotAv')
        return self

    def transform(self, X):
        for key, value in self.values.items():
            if key in X.columns:
                X[key] = X[key].fillna(value)

        for c in self.cat_cols:
            if c in X.columns:  
                if X[c].dtype in ['int64', 'float64']:
                    X[c] = X[c].astype(str)
                X[c] = X[c].fillna('NotAv')

        return X

# Feature Selection

In [84]:
class BoostedFeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(
        self,
        target_feature_count=150,
        xgb_trees=100,
        tree_depth=6,
        lr=0.05,
        row_sampling=0.8,
        col_sampling=0.8,
        parallel_jobs=-1,
        seed=42,
        silent=True
    ):
      
        self.target_feature_count = target_feature_count
        self.xgb_trees = xgb_trees
        self.tree_depth = tree_depth
        self.lr = lr
        self.row_sampling = row_sampling
        self.col_sampling = col_sampling
        self.parallel_jobs = parallel_jobs
        self.seed = seed
        self.silent = silent
        self.kept_features_ = None
        self.feature_mask_ = None
        self.feature_ranks_ = None

    def fit(self, X, y):
       
        
        xgb_model = XGBRegressor(
            n_estimators=self.xgb_trees,
            max_depth=self.tree_depth,
            learning_rate=self.lr,
            subsample=self.row_sampling,
            colsample_bytree=self.col_sampling,
            n_jobs=self.parallel_jobs,
            random_state=self.seed,
            verbosity=0 if self.silent else 1,
            tree_method='hist',
            objective='reg:squarederror'
        )
        
        if X.shape[1] > 200:
           
            rough_selector = RFE(
                estimator=xgb_model,
                n_features_to_select=min(200, X.shape[1]),
                step=max(1, X.shape[1] // 20)  
            )
            rough_selector.fit(X, y)
            X = X.loc[:, rough_selector.support_]
        
        final_selector = RFE(
            estimator=xgb_model,
            n_features_to_select=self.target_feature_count,
            step=1 
        )
        final_selector.fit(X, y)
        
        if hasattr(X, 'columns'):
            self.kept_features_ = X.columns[final_selector.support_].tolist()
        self.feature_mask_ = final_selector.support_
        self.feature_ranks_ = final_selector.ranking_
        
        print(f"Done! Selected {len(self.kept_features_)} features.")
        return self

    def transform(self, X):
       
        if self.kept_features_ is None:
            raise RuntimeError("Fit the selector first!")
        
        if isinstance(X, pd.DataFrame):
            return X[self.kept_features_]
        return X[:, self.feature_mask_]

    def get_feature_indices(self, as_indices=False):
       
        return np.where(self.feature_mask_)[0] if as_indices else self.feature_mask_

# Training

In [119]:
name = "Fraud_detection_XGBoost"
mlflow.set_experiment(name)
# pipeline Transformator parameters
drop_threshold=0.75
sampling_strategy =0.2
correlation_threshold = 0.8
feature_count = 60
prediction_threshold = 0.7

# Classifier parameters
n_estimators = 500
max_depth = 10
learning_rate = 0.1
subsample=0.8
colsample_bytree=0.8  
random_state=42
n_jobs=-1
reg_lambda=3    
reg_alpha=0.1     
gamma=2        
use_label_encoder=False
eval_metric='logloss'


imb_pipeline = ImbPipeline([
        ('col_dropper', DropColumns(threshold=drop_threshold)),
        ('undersampler',RandomUnderSampler(random_state=42, sampling_strategy=sampling_strategy)),
        ('fill_na',FillNa(num_cols=num_columns, cat_cols=cat_columns)),
        ('encoder',CatEncoder()),
        ('corr_filter',CorrelationTargetFilter(threshold=correlation_threshold)),
        ('feature_selector', BoostedFeatureSelector(
        target_feature_count=feature_count,
        xgb_trees=n_estimators,
        tree_depth=max_depth,
        lr=learning_rate
    )),
        ('classifier',XGBClassifier(n_estimators=n_estimators, 
        max_depth=max_depth,      
        learning_rate=learning_rate,
        subsample=subsample,
        colsample_bytree=colsample_bytree,  
        random_state=random_state,
        n_jobs=n_jobs,
        reg_lambda=reg_lambda,      
        reg_alpha=reg_alpha,     
        gamma=gamma,           
        use_label_encoder=use_label_encoder,
        eval_metric=eval_metric))
])

with mlflow.start_run(run_name="XGB 25", nested=True) as run:
    imb_pipeline.fit(X_train,y_train)
    
    mlflow.sklearn.log_model(imb_pipeline, "imb_XGBoost")
    xgb_model = imb_pipeline.named_steps["classifier"]
    mlflow.log_params({
        "column_drop_threshold" : drop_threshold,
        "sampling_strategy": sampling_strategy,
        "correlation_drop_threshold":correlation_threshold,
        "n_features": feature_count,
        "n_estimators": xgb_model.n_estimators,
        "max_depth": xgb_model.max_depth,
        "learning_rate": xgb_model.learning_rate,
        "subsample": xgb_model.subsample,
        "colsample_bytree": xgb_model.colsample_bytree,
        "random_state": xgb_model.random_state,
        "reg_alpha": xgb_model.reg_alpha,
    })

    probas_train = imb_pipeline.predict_proba(X_train)[:, 1]
    preds_train = (probas_train >= prediction_threshold).astype(int)
    
    probas = imb_pipeline.predict_proba(X_test)[:, 1]
    preds = (probas >= prediction_threshold).astype(int)
    f1_train = f1_score(y_train, preds_train)

    # metrics
    prec = precision_score(y_test, preds)
    rec = recall_score(y_test, preds)
    f1 = f1_score(y_test, preds)
    auc = roc_auc_score(y_test, probas)

    split='test'
    mlflow.log_metrics({
            f"{split}_precision": prec,
            f"{split}_recall": rec,
            f"{split}_f1_score": f1,
            f"{split}_auc": auc,
            f"{split}_threshold": prediction_threshold
        })


    # ROC curve
    fpr, tpr, _ = roc_curve(y_test, probas)
    plt.figure()
    plt.plot(fpr, tpr, label=f"AUC = {auc:.3f}")
    plt.plot([0, 1], [0, 1], linestyle='--')
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"{split.capitalize()} ROC Curve")
    plt.legend()
    roc_path = f"{split}_roc_curve.png"
    plt.savefig(roc_path)
    plt.close()
    mlflow.log_artifact(roc_path)
    
    

Found 133 features with high correlation to drop.
Done! Selected 60 features.


2025/05/01 19:31:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGB 25 at: https://dagshub.com/gchit21/ML2.mlflow/#/experiments/1/runs/caec2bf33555488e8939e1025161a0ff
🧪 View experiment at: https://dagshub.com/gchit21/ML2.mlflow/#/experiments/1


In [120]:
f1_train

0.6597842323651453